In [96]:
import sys
from sklearn.model_selection import KFold

from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, concatenate
from keras_contrib.layers import CRF
from keras.optimizers import RMSprop

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import ast


In [97]:

def instances(fi):
    xseq = []
    yseq = []

    for line in fi:
        line = line.strip('\n')
        if not line:
            # An empty line means the end of a sentence.
            # Return accumulated sequences, and reinitialize.
            yield xseq, yseq
            xseq = []
            yseq = []
            continue

        # Split the line with TAB characters.
        fields = line.split('\t')

        # Append the item features to the item sequence.
        # fields are:  0=sid, 1=id_e1, 2=id_e2, 3=interaction, 4=tag, 5...N = features
        item = fields[5:]
        xseq.append(item)

        # Append the label to the label sequence.
        yseq.append(fields[4])
    #return xseq, yseq
    
def get_sentences(fi):

    for line in fi:
        line = line.strip('\n')        
        if not line:
            continue
        # Split the line with TAB characters.
        fields = line.split('\t')
        yield fields[5], fields[4]

def load_data(fi):
    xtrain = []
    ytrain = []

    for line in fi:
        line = line.strip('\n')
        if not line:
            continue
        # Split the line with TAB characters.
        fields = line.split('\t')

        # Append the item features to the item sequence.
        # fields are:  0=sid, 1=id_e1, 2=id_e2, 3=interaction, 4=tag, 5...N = features
        item = fields[5:]
        xtrain.append(item)

        # Append the label to the label sequence.
        ytrain.append(fields[4])
    return xtrain, ytrain



def instances_pred(fi):
    xseq = []
    toks = []

    for line in fi:
        line = line.strip('\n')
        if not line:
            # An empty line means the end of a sentence.
            # Return accumulated sequences, and reinitialize.
            yield xseq, toks
            xseq = []
            toks = []
            continue

        # Split the line with TAB characters.
        fields = line.split('\t')

        # Append the item features to the item sequence.
        # fields are:  0=sid, 1=form, 2=span_start, 3=span_end, 4=tag, 5...N = features
        # fields are:  0=sid, 1=id_e1, 2=id_e2, 3=interaction, 4=tag, 5...N = features
        item = fields[5:]
        xseq.append(item)

        # Append the label to the label sequence.
        toks.append([fields[0],fields[1],fields[2]])

def load_glove():
    # load the whole embedding into memory
    embeddings_index = dict()
    f = open('../../../data/glove.6B/glove.6B.100d.txt')
    for line in f:
    	values = line.split()
    	word = values[0]
    	coefs = np.asarray(values[1:], dtype='float32')
    	embeddings_index[word] = coefs
    f.close()
    return embeddings_index


In [98]:
sentences = []
tags = []

fi = open('results/2/train.cod')

# Read sentences from STDIN, and append them to the trainer.
for xseq, yseq in get_sentences(fi):
    sentences.append(ast.literal_eval(xseq))
    tags.append(yseq)
fi.close()
#fi = open('results/2/train.cod')
#(xtrain, ytrain) = load_data(fi)
x = pd.DataFrame(sentences)
y = pd.DataFrame(tags).values[:,0]

max_len = 75
words = list(set(x[0].values))
words.append("ENDPAD")
n_words = len(words)
utags = list(set(y))
n_tags = len(utags)

word2idx = {w: i + 1 for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(utags)}

docs = [[w[0] for w in s] for s in sentences]
t = Tokenizer(oov_token='<unw>')
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
encoded_docs = t.texts_to_sequences(docs)
X = pad_sequences(maxlen=max_len, sequences=encoded_docs, padding="post", value=vocab_size-1)

embeddings_index = load_glove()
embedding_matrix = np.zeros((vocab_size, 100))
#print(t.word_index.items())
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        
Y = [[tag2idx[tag] for tag in tags]]
#Y = pad_sequences(maxlen=max_len, sequences=Y, padding="post", value=tag2idx["null"])
#Y = [to_categorical(i, num_classes=n_tags) for i in Y]


In [113]:
Y = [[tag2idx[tag]] for tag in tags]
Y = pad_sequences(maxlen=max_len, sequences=Y, padding="post", value=tag2idx["null"])
Y = [to_categorical(i, num_classes=n_tags) for i in Y]

In [114]:
np.array(Y).shape

(26005, 75, 5)

In [115]:
word_in = Input(shape=(max_len,))
word_emb = Embedding(input_dim=vocab_size, output_dim=100,
                  input_length=max_len, mask_zero=True, weights=[embedding_matrix],
                   trainable=False)(word_in)  # 20-dim embedding
model = Bidirectional(LSTM(units=32, return_sequences=True,
                           recurrent_dropout=0.5))(word_emb)  # variational biLSTM
model = TimeDistributed(Dense(20, activation="relu"))(model)  # a dense layer as suggested by neuralNer
crf = CRF(n_tags)  # CRF layer
out = crf(model)  # output
model = Model(word_in, out)

In [119]:
optimizer = RMSprop(lr=0.01, rho=0.9, epsilon=None, decay=0.0)
model.compile(optimizer=optimizer, loss=crf.loss_function, metrics=[crf.accuracy])
model.fit(X, np.array(Y), validation_split=0.1, verbose=1, epochs=5)

/home/santiago/.local/lib/python3.6/site-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/home/santiago/.local/lib/python3.6/site-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


Train on 23404 samples, validate on 2601 samples
Epoch 1/5
23404/23404 [==============================] - 75s 3ms/step - loss: 0.0075 - crf_viterbi_accuracy: 0.9980 - val_loss: 0.0091 - val_crf_viterbi_accuracy: 0.9982
Epoch 2/5
23404/23404 [==============================] - 72s 3ms/step - loss: 0.0075 - crf_viterbi_accuracy: 0.9980 - val_loss: 0.0101 - val_crf_viterbi_accuracy: 0.9982
Epoch 3/5
23404/23404 [==============================] - 68s 3ms/step - loss: 0.0075 - crf_viterbi_accuracy: 0.9980 - val_loss: 0.0114 - val_crf_viterbi_accuracy: 0.9982
Epoch 4/5
23404/23404 [==============================] - 67s 3ms/step - loss: 0.0074 - crf_viterbi_accuracy: 0.9980 - val_loss: 0.0166 - val_crf_viterbi_accuracy: 0.9982
Epoch 5/5
23404/23404 [==============================] - 66s 3ms/step - loss: 0.0077 - crf_viterbi_accuracy: 0.9980 - val_loss: 0.0178 - val_crf_viterbi_accuracy: 0.9982


In [120]:
for xseq,toks in instances_pred(open('results/1/test.cod')):
    #x = pd.DataFrame(xseq)
    #x_test_sent = [[word2idx.get(t[0],0) for t in xseq]]
    docs = [[t[0] for t in xseq]]
    #pos = [[pos2idx.get(t[4],0) for t in xseq]]
    #suff3 = [[suff32idx.get(t[2],0) for t in xseq]]
    #suff4 = [[suff42idx.get(t[3],0) for t in xseq]]
    #tktypes = [[tktypes2idx.get(t[5],0) for t in xseq]]
    encoded_docs = t.texts_to_sequences(docs)
    x_test_sent = pad_sequences(maxlen=max_len, sequences=encoded_docs, padding="post", value=vocab_size-1)
    #x_test_pos = pad_sequences(maxlen=max_len, sequences=pos, padding="post", value=n_pos-1)
    #x_test_suff3 = pad_sequences(maxlen=max_len, sequences=suff3, padding="post", value=n_suff3-1)
    #x_test_suff4 = pad_sequences(maxlen=max_len, sequences=suff4, padding="post", value=n_suff4-1)
    #x_test_tktypes = pad_sequences(maxlen=max_len, sequences=tktypes, padding="post", value=n_tktypes-1)

    #prediction = model.predict([x_test_sent, x_test_pos,
    #x_test_suff3,
    #x_test_suff4,
    #x_test_tktypes])
    prediction = model.predict(x_test_sent)[0]
    prediction = np.argmax(prediction, axis=-1)
    inside = False;
    for k in range(0,len(toks)) :
        #prediction_max = np.argmax(prediction[k], axis=-1)
        #y = utags[prediction_max[k]]
        #y = utags[prediction[k]]
        #(sid, form, offS, offE) = toks[k]
        #y = prediction[k]
        y = utags[prediction[k]]
        (sid, id_e1, id_e2) = toks[k]
        if y == "null":
            int = "0"
        else:
            int = "1"
        print(sid,id_e1,id_e2,int,y, sep="|")

DDI-DrugBank.d757.s0|DDI-DrugBank.d757.s0.e0|DDI-DrugBank.d757.s0.e1|0|null
DDI-DrugBank.d757.s0|DDI-DrugBank.d757.s0.e0|DDI-DrugBank.d757.s0.e2|0|null
DDI-DrugBank.d757.s0|DDI-DrugBank.d757.s0.e1|DDI-DrugBank.d757.s0.e2|0|null
DDI-DrugBank.d765.s0|DDI-DrugBank.d765.s0.e0|DDI-DrugBank.d765.s0.e1|0|null
DDI-DrugBank.d765.s1|DDI-DrugBank.d765.s1.e0|DDI-DrugBank.d765.s1.e1|0|null
DDI-DrugBank.d765.s1|DDI-DrugBank.d765.s1.e0|DDI-DrugBank.d765.s1.e2|0|null
DDI-DrugBank.d765.s1|DDI-DrugBank.d765.s1.e0|DDI-DrugBank.d765.s1.e3|0|null
DDI-DrugBank.d765.s1|DDI-DrugBank.d765.s1.e0|DDI-DrugBank.d765.s1.e4|0|null
DDI-DrugBank.d765.s1|DDI-DrugBank.d765.s1.e0|DDI-DrugBank.d765.s1.e5|0|null
DDI-DrugBank.d765.s1|DDI-DrugBank.d765.s1.e0|DDI-DrugBank.d765.s1.e6|0|null
DDI-DrugBank.d765.s1|DDI-DrugBank.d765.s1.e0|DDI-DrugBank.d765.s1.e7|0|null
DDI-DrugBank.d765.s1|DDI-DrugBank.d765.s1.e0|DDI-DrugBank.d765.s1.e8|0|null
DDI-DrugBank.d765.s1|DDI-DrugBank.d765.s1.e0|DDI-DrugBank.d765.s1.e9|0|null
DDI-DrugBank

DDI-DrugBank.d675.s2|DDI-DrugBank.d675.s2.e3|DDI-DrugBank.d675.s2.e12|0|null
DDI-DrugBank.d675.s2|DDI-DrugBank.d675.s2.e3|DDI-DrugBank.d675.s2.e13|0|null
DDI-DrugBank.d675.s2|DDI-DrugBank.d675.s2.e4|DDI-DrugBank.d675.s2.e5|0|null
DDI-DrugBank.d675.s2|DDI-DrugBank.d675.s2.e4|DDI-DrugBank.d675.s2.e6|0|null
DDI-DrugBank.d675.s2|DDI-DrugBank.d675.s2.e4|DDI-DrugBank.d675.s2.e7|0|null
DDI-DrugBank.d675.s2|DDI-DrugBank.d675.s2.e4|DDI-DrugBank.d675.s2.e8|0|null
DDI-DrugBank.d675.s2|DDI-DrugBank.d675.s2.e4|DDI-DrugBank.d675.s2.e9|0|null
DDI-DrugBank.d675.s2|DDI-DrugBank.d675.s2.e4|DDI-DrugBank.d675.s2.e10|0|null
DDI-DrugBank.d675.s2|DDI-DrugBank.d675.s2.e4|DDI-DrugBank.d675.s2.e11|0|null
DDI-DrugBank.d675.s2|DDI-DrugBank.d675.s2.e4|DDI-DrugBank.d675.s2.e12|0|null
DDI-DrugBank.d675.s2|DDI-DrugBank.d675.s2.e4|DDI-DrugBank.d675.s2.e13|0|null
DDI-DrugBank.d675.s2|DDI-DrugBank.d675.s2.e5|DDI-DrugBank.d675.s2.e6|0|null
DDI-DrugBank.d675.s2|DDI-DrugBank.d675.s2.e5|DDI-DrugBank.d675.s2.e7|0|null
DDI-Dr

DDI-DrugBank.d633.s15|DDI-DrugBank.d633.s15.e1|DDI-DrugBank.d633.s15.e4|0|null
DDI-DrugBank.d633.s15|DDI-DrugBank.d633.s15.e2|DDI-DrugBank.d633.s15.e3|0|null
DDI-DrugBank.d633.s15|DDI-DrugBank.d633.s15.e2|DDI-DrugBank.d633.s15.e4|0|null
DDI-DrugBank.d633.s15|DDI-DrugBank.d633.s15.e3|DDI-DrugBank.d633.s15.e4|0|null
DDI-DrugBank.d633.s16|DDI-DrugBank.d633.s16.e0|DDI-DrugBank.d633.s16.e1|0|null
DDI-DrugBank.d633.s16|DDI-DrugBank.d633.s16.e0|DDI-DrugBank.d633.s16.e2|0|null
DDI-DrugBank.d633.s16|DDI-DrugBank.d633.s16.e1|DDI-DrugBank.d633.s16.e2|0|null
DDI-DrugBank.d633.s18|DDI-DrugBank.d633.s18.e0|DDI-DrugBank.d633.s18.e1|0|null
DDI-DrugBank.d633.s18|DDI-DrugBank.d633.s18.e0|DDI-DrugBank.d633.s18.e2|0|null
DDI-DrugBank.d633.s18|DDI-DrugBank.d633.s18.e0|DDI-DrugBank.d633.s18.e3|0|null
DDI-DrugBank.d633.s18|DDI-DrugBank.d633.s18.e0|DDI-DrugBank.d633.s18.e4|0|null
DDI-DrugBank.d633.s18|DDI-DrugBank.d633.s18.e0|DDI-DrugBank.d633.s18.e5|0|null
DDI-DrugBank.d633.s18|DDI-DrugBank.d633.s18.e0|DDI-D

DDI-DrugBank.d737.s6|DDI-DrugBank.d737.s6.e4|DDI-DrugBank.d737.s6.e11|0|null
DDI-DrugBank.d737.s6|DDI-DrugBank.d737.s6.e4|DDI-DrugBank.d737.s6.e12|0|null
DDI-DrugBank.d737.s6|DDI-DrugBank.d737.s6.e4|DDI-DrugBank.d737.s6.e13|0|null
DDI-DrugBank.d737.s6|DDI-DrugBank.d737.s6.e5|DDI-DrugBank.d737.s6.e6|0|null
DDI-DrugBank.d737.s6|DDI-DrugBank.d737.s6.e5|DDI-DrugBank.d737.s6.e7|0|null
DDI-DrugBank.d737.s6|DDI-DrugBank.d737.s6.e5|DDI-DrugBank.d737.s6.e8|0|null
DDI-DrugBank.d737.s6|DDI-DrugBank.d737.s6.e5|DDI-DrugBank.d737.s6.e9|0|null
DDI-DrugBank.d737.s6|DDI-DrugBank.d737.s6.e5|DDI-DrugBank.d737.s6.e10|0|null
DDI-DrugBank.d737.s6|DDI-DrugBank.d737.s6.e5|DDI-DrugBank.d737.s6.e11|0|null
DDI-DrugBank.d737.s6|DDI-DrugBank.d737.s6.e5|DDI-DrugBank.d737.s6.e12|0|null
DDI-DrugBank.d737.s6|DDI-DrugBank.d737.s6.e5|DDI-DrugBank.d737.s6.e13|0|null
DDI-DrugBank.d737.s6|DDI-DrugBank.d737.s6.e6|DDI-DrugBank.d737.s6.e7|0|null
DDI-DrugBank.d737.s6|DDI-DrugBank.d737.s6.e6|DDI-DrugBank.d737.s6.e8|0|null
DDI-D

DDI-DrugBank.d618.s1|DDI-DrugBank.d618.s1.e3|DDI-DrugBank.d618.s1.e5|0|null
DDI-DrugBank.d618.s1|DDI-DrugBank.d618.s1.e4|DDI-DrugBank.d618.s1.e5|0|null
DDI-DrugBank.d783.s0|DDI-DrugBank.d783.s0.e0|DDI-DrugBank.d783.s0.e1|0|null
DDI-DrugBank.d783.s0|DDI-DrugBank.d783.s0.e0|DDI-DrugBank.d783.s0.e2|0|null
DDI-DrugBank.d783.s0|DDI-DrugBank.d783.s0.e0|DDI-DrugBank.d783.s0.e3|0|null
DDI-DrugBank.d783.s0|DDI-DrugBank.d783.s0.e0|DDI-DrugBank.d783.s0.e4|0|null
DDI-DrugBank.d783.s0|DDI-DrugBank.d783.s0.e0|DDI-DrugBank.d783.s0.e5|0|null
DDI-DrugBank.d783.s0|DDI-DrugBank.d783.s0.e1|DDI-DrugBank.d783.s0.e2|0|null
DDI-DrugBank.d783.s0|DDI-DrugBank.d783.s0.e1|DDI-DrugBank.d783.s0.e3|0|null
DDI-DrugBank.d783.s0|DDI-DrugBank.d783.s0.e1|DDI-DrugBank.d783.s0.e4|0|null
DDI-DrugBank.d783.s0|DDI-DrugBank.d783.s0.e1|DDI-DrugBank.d783.s0.e5|0|null
DDI-DrugBank.d783.s0|DDI-DrugBank.d783.s0.e2|DDI-DrugBank.d783.s0.e3|0|null
DDI-DrugBank.d783.s0|DDI-DrugBank.d783.s0.e2|DDI-DrugBank.d783.s0.e4|0|null
DDI-DrugBank

...

DDI-DrugBank.d656.s2|DDI-DrugBank.d656.s2.e0|DDI-DrugBank.d656.s2.e2|0|null
DDI-DrugBank.d656.s2|DDI-DrugBank.d656.s2.e0|DDI-DrugBank.d656.s2.e3|0|null
DDI-DrugBank.d656.s2|DDI-DrugBank.d656.s2.e0|DDI-DrugBank.d656.s2.e4|0|null
DDI-DrugBank.d656.s2|DDI-DrugBank.d656.s2.e0|DDI-DrugBank.d656.s2.e5|0|null
DDI-DrugBank.d656.s2|DDI-DrugBank.d656.s2.e0|DDI-DrugBank.d656.s2.e6|0|null
DDI-DrugBank.d656.s2|DDI-DrugBank.d656.s2.e0|DDI-DrugBank.d656.s2.e7|0|null
DDI-DrugBank.d656.s2|DDI-DrugBank.d656.s2.e1|DDI-DrugBank.d656.s2.e2|0|null
DDI-DrugBank.d656.s2|DDI-DrugBank.d656.s2.e1|DDI-DrugBank.d656.s2.e3|0|null
DDI-DrugBank.d656.s2|DDI-DrugBank.d656.s2.e1|DDI-DrugBank.d656.s2.e4|0|null
DDI-DrugBank.d656.s2|DDI-DrugBank.d656.s2.e1|DDI-DrugBank.d656.s2.e5|0|null
DDI-DrugBank.d656.s2|DDI-DrugBank.d656.s2.e1|DDI-DrugBank.d656.s2.e6|0|null
DDI-DrugBank.d656.s2|DDI-DrugBank.d656.s2.e1|DDI-DrugBank.d656.s2.e7|0|null
DDI-DrugBank.d656.s2|DDI-DrugBank.d656.s2.e2|DDI-DrugBank.d656.s2.e3|0|null
DDI-DrugBank

DDI-DrugBank.d739.s10|DDI-DrugBank.d739.s10.e5|DDI-DrugBank.d739.s10.e7|0|null
DDI-DrugBank.d739.s10|DDI-DrugBank.d739.s10.e5|DDI-DrugBank.d739.s10.e8|0|null
DDI-DrugBank.d739.s10|DDI-DrugBank.d739.s10.e5|DDI-DrugBank.d739.s10.e9|0|null
DDI-DrugBank.d739.s10|DDI-DrugBank.d739.s10.e6|DDI-DrugBank.d739.s10.e7|0|null
DDI-DrugBank.d739.s10|DDI-DrugBank.d739.s10.e6|DDI-DrugBank.d739.s10.e8|0|null
DDI-DrugBank.d739.s10|DDI-DrugBank.d739.s10.e6|DDI-DrugBank.d739.s10.e9|0|null
DDI-DrugBank.d739.s10|DDI-DrugBank.d739.s10.e7|DDI-DrugBank.d739.s10.e8|0|null
DDI-DrugBank.d739.s10|DDI-DrugBank.d739.s10.e7|DDI-DrugBank.d739.s10.e9|0|null
DDI-DrugBank.d739.s10|DDI-DrugBank.d739.s10.e8|DDI-DrugBank.d739.s10.e9|0|null
DDI-DrugBank.d739.s12|DDI-DrugBank.d739.s12.e0|DDI-DrugBank.d739.s12.e1|0|null
DDI-DrugBank.d739.s12|DDI-DrugBank.d739.s12.e0|DDI-DrugBank.d739.s12.e2|0|null
DDI-DrugBank.d739.s12|DDI-DrugBank.d739.s12.e0|DDI-DrugBank.d739.s12.e3|0|null
DDI-DrugBank.d739.s12|DDI-DrugBank.d739.s12.e0|DDI-D

DDI-DrugBank.d585.s0|DDI-DrugBank.d585.s0.e3|DDI-DrugBank.d585.s0.e4|0|null
DDI-DrugBank.d585.s0|DDI-DrugBank.d585.s0.e3|DDI-DrugBank.d585.s0.e5|0|null
DDI-DrugBank.d585.s0|DDI-DrugBank.d585.s0.e3|DDI-DrugBank.d585.s0.e6|0|null
DDI-DrugBank.d585.s0|DDI-DrugBank.d585.s0.e3|DDI-DrugBank.d585.s0.e7|0|null
DDI-DrugBank.d585.s0|DDI-DrugBank.d585.s0.e3|DDI-DrugBank.d585.s0.e8|0|null
DDI-DrugBank.d585.s0|DDI-DrugBank.d585.s0.e3|DDI-DrugBank.d585.s0.e9|0|null
DDI-DrugBank.d585.s0|DDI-DrugBank.d585.s0.e3|DDI-DrugBank.d585.s0.e10|0|null
DDI-DrugBank.d585.s0|DDI-DrugBank.d585.s0.e3|DDI-DrugBank.d585.s0.e11|0|null
DDI-DrugBank.d585.s0|DDI-DrugBank.d585.s0.e3|DDI-DrugBank.d585.s0.e12|0|null
DDI-DrugBank.d585.s0|DDI-DrugBank.d585.s0.e3|DDI-DrugBank.d585.s0.e13|0|null
DDI-DrugBank.d585.s0|DDI-DrugBank.d585.s0.e3|DDI-DrugBank.d585.s0.e14|0|null
DDI-DrugBank.d585.s0|DDI-DrugBank.d585.s0.e4|DDI-DrugBank.d585.s0.e5|0|null
DDI-DrugBank.d585.s0|DDI-DrugBank.d585.s0.e4|DDI-DrugBank.d585.s0.e6|0|null
DDI-Dru

DDI-DrugBank.d601.s0|DDI-DrugBank.d601.s0.e1|DDI-DrugBank.d601.s0.e7|0|null
DDI-DrugBank.d601.s0|DDI-DrugBank.d601.s0.e2|DDI-DrugBank.d601.s0.e3|0|null
DDI-DrugBank.d601.s0|DDI-DrugBank.d601.s0.e2|DDI-DrugBank.d601.s0.e4|0|null
DDI-DrugBank.d601.s0|DDI-DrugBank.d601.s0.e2|DDI-DrugBank.d601.s0.e5|0|null
DDI-DrugBank.d601.s0|DDI-DrugBank.d601.s0.e2|DDI-DrugBank.d601.s0.e6|0|null
DDI-DrugBank.d601.s0|DDI-DrugBank.d601.s0.e2|DDI-DrugBank.d601.s0.e7|0|null
DDI-DrugBank.d601.s0|DDI-DrugBank.d601.s0.e3|DDI-DrugBank.d601.s0.e4|0|null
DDI-DrugBank.d601.s0|DDI-DrugBank.d601.s0.e3|DDI-DrugBank.d601.s0.e5|0|null
DDI-DrugBank.d601.s0|DDI-DrugBank.d601.s0.e3|DDI-DrugBank.d601.s0.e6|0|null
DDI-DrugBank.d601.s0|DDI-DrugBank.d601.s0.e3|DDI-DrugBank.d601.s0.e7|0|null
DDI-DrugBank.d601.s0|DDI-DrugBank.d601.s0.e4|DDI-DrugBank.d601.s0.e5|0|null
DDI-DrugBank.d601.s0|DDI-DrugBank.d601.s0.e4|DDI-DrugBank.d601.s0.e6|0|null
DDI-DrugBank.d601.s0|DDI-DrugBank.d601.s0.e4|DDI-DrugBank.d601.s0.e7|0|null
DDI-DrugBank

DDI-DrugBank.d711.s2|DDI-DrugBank.d711.s2.e0|DDI-DrugBank.d711.s2.e5|0|null
DDI-DrugBank.d711.s2|DDI-DrugBank.d711.s2.e1|DDI-DrugBank.d711.s2.e2|0|null
DDI-DrugBank.d711.s2|DDI-DrugBank.d711.s2.e1|DDI-DrugBank.d711.s2.e3|0|null
DDI-DrugBank.d711.s2|DDI-DrugBank.d711.s2.e1|DDI-DrugBank.d711.s2.e4|0|null
DDI-DrugBank.d711.s2|DDI-DrugBank.d711.s2.e1|DDI-DrugBank.d711.s2.e5|0|null
DDI-DrugBank.d711.s2|DDI-DrugBank.d711.s2.e2|DDI-DrugBank.d711.s2.e3|0|null
DDI-DrugBank.d711.s2|DDI-DrugBank.d711.s2.e2|DDI-DrugBank.d711.s2.e4|0|null
DDI-DrugBank.d711.s2|DDI-DrugBank.d711.s2.e2|DDI-DrugBank.d711.s2.e5|0|null
DDI-DrugBank.d711.s2|DDI-DrugBank.d711.s2.e3|DDI-DrugBank.d711.s2.e4|0|null
DDI-DrugBank.d711.s2|DDI-DrugBank.d711.s2.e3|DDI-DrugBank.d711.s2.e5|0|null
DDI-DrugBank.d711.s2|DDI-DrugBank.d711.s2.e4|DDI-DrugBank.d711.s2.e5|0|null
DDI-DrugBank.d711.s3|DDI-DrugBank.d711.s3.e0|DDI-DrugBank.d711.s3.e1|0|null
DDI-DrugBank.d711.s4|DDI-DrugBank.d711.s4.e0|DDI-DrugBank.d711.s4.e1|0|null
DDI-DrugBank

In [17]:
prediction

array([[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3]])

In [94]:
np.array(Y[0]).shape

(26005,)

In [88]:
import ast
ast.literal_eval(sentences[0])



ValueError: malformed node or string: ['the', 'potential', 'for', 'increased', 'sedation', 'when', 'guanfacine', 'is', 'given', 'with', 'other', 'cns-depressant', 'drug', 'should', 'be', 'appreciated', '.']

In [91]:
Y = [tag2idx[tag] for tag in tags]
#Y = pad_sequences(maxlen=max_len, sequences=Y, padding="post", value=tag2idx["null"])
np.array(Y).shape

(26005,)